In [9]:
#import all helpers
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
import seaborn as sns # More snazzy plotting library
import itertools
##impot classifieris
#-----Ensemble---------------------
from sklearn.ensemble import       AdaBoostClassifier
from sklearn.ensemble import       BaggingClassifier
from sklearn.ensemble import       ExtraTreesClassifier
from sklearn.ensemble import       GradientBoostingClassifier
from sklearn.ensemble import       RandomForestClassifier

#----Generalized Linear models-----
from sklearn.linear_model import   PassiveAggressiveClassifier
from sklearn.linear_model import   LogisticRegression
from sklearn.linear_model import   RidgeClassifier
from sklearn.linear_model import   SGDClassifier

#-----Naive Bayes ---all class-----
from sklearn.naive_bayes import    GaussianNB
from sklearn.naive_bayes import    MultinomialNB
from sklearn.naive_bayes import    BernoulliNB

#---Nearest Neighbors--------------
from sklearn.neighbors import      KNeighborsClassifier
from sklearn.neighbors import      RadiusNeighborsClassifier
from sklearn.neighbors import      NearestCentroid

#----Neural Networks---------------
from sklearn.neural_network import MLPClassifier

#-----Support Vector Machines------
from sklearn.svm import            SVC
from sklearn.svm import            LinearSVC
from sklearn.svm import            NuSVC

#-----Decission Trees--------------
from sklearn.tree import           DecisionTreeClassifier
from sklearn.tree import           ExtraTreeClassifier


#file_path =  "../dataset/movie_metadata_cleaned_tfidf_num_only_min.csv"
file_path =  "../dataset/movie_metadata_cleaned_categ_num_only.csv"
#file_path = "../dataset/movie_metadata_cleaned_no_vector_num_only.csv"

dta = pd.read_csv(file_path)
dta_clean = dta
#remove the null values, that is fill NaN with there - FIXME: Rihards, naive implementation
dta_clean = dta_clean.fillna(value=0, axis=1)
dta_clean = dta_clean.dropna()
dta_clean = dta_clean.drop('Unnamed: 0', axis=1)
dta_clean.describe()

#clasify the data for the logistic regression
#def label_gross (gross):
#    if   (gross <= 10000000) : return 0
#    elif (10000000 < gross < 100000000) : return 1
#    else : return 2
    
def label_gross (gross):
    if (gross < 1000000) : return 1
    elif ((gross >= 1000000) & (gross < 10000000)) : return 2
    elif ((gross >= 10000000) & (gross < 50000000)) : return 3
    elif ((gross >= 50000000) & (gross < 200000000)) : return 4
    elif (gross >= 200000000) : return 5
    
y = dta_clean.worldwide_gross.apply (lambda gross: label_gross (gross))
X = dta_clean.drop('worldwide_gross', axis=1)
X, _X_dummy, y, _y_dummy = train_test_split(X, y, test_size=0)

#y = pd.DataFrame(y)
#class0 = y[y['worldwide_gross'] == 1]
#class1 = y[y['worldwide_gross'] == 2]
#class2 = y[y['worldwide_gross'] == 3]
#class3 = y[y['worldwide_gross'] == 4]
#class4 = y[y['worldwide_gross'] == 5]

In [13]:
models = [AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,RandomForestClassifier,PassiveAggressiveClassifier,LogisticRegression,RidgeClassifier,SGDClassifier,GaussianNB,MultinomialNB,BernoulliNB,KNeighborsClassifier,RadiusNeighborsClassifier,NearestCentroid,MLPClassifier,SVC,LinearSVC,NuSVC,DecisionTreeClassifier,ExtraTreeClassifier]
#models = [GradientBoostingClassifier]
results = {}
errors = {}

def run_grid_search(x,y,Model, res_dict, error):
    #create pipline and use GridSearch to find the betst params
    name = Model.__name__
    #if name == "MLPClassifier" or name == "MLPRegressor": 
    #    cv = 50
    #else: 
    cv = 5
    model = Model()
    pipe = Pipeline(steps=[('model', model)])
    #create estimator
    print()
    print ("***Starting ["  + name + "] estimator run ")
    estimator = GridSearchCV(pipe,dict(),verbose=2, cv=cv, n_jobs=4)
    #run the esmimator, except eceptions, sape errors
    try:
            estimator.fit(x, y)
            print ("GREP_ME***Results of ["  + name + "] estimator run are")
            print (estimator.cv_results_)
            print ("GREP_ME***Best params of ["  + name + "] estimator run are")
            print (estimator.best_params_)
            print ("GREP_ME***Best score of ["  + name + "] estimator run are")
            print (estimator.best_score_)
            results[name] = estimator.best_score_
    except ValueError as err:
            print ("GREP_ME***Error caught for  ["  + name + "]")
            errors[name] = err
            pass
            
def run_solver(X,y,models_arr, res_dict, errors):
    for model in models_arr:
        run_grid_search(X,y, model, res_dict, errors)       

In [15]:
run_solver(X,y, models, results, errors)


***Starting [AdaBoostClassifier] estimator run 
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV] ................................................. , total=   0.5s
[CV] ................................................. , total=   0.5s
[CV] ................................................. , total=   0.5s


[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    1.2s finished


GREP_ME***Results of [AdaBoostClassifier] estimator run are
{'std_score_time': array([ 0.00080224]), 'params': ({},), 'split0_test_score': array([ 0.56891192]), 'split4_train_score': array([ 0.56334372]), 'std_train_score': array([ 0.01294753]), 'split4_test_score': array([ 0.52083333]), 'mean_score_time': array([ 0.02453418]), 'split3_train_score': array([ 0.54531291]), 'split0_train_score': array([ 0.55133871]), 'split2_test_score': array([ 0.49376299]), 'split1_train_score': array([ 0.56418919]), 'split2_train_score': array([ 0.52909091]), 'std_test_score': array([ 0.02924816]), 'std_fit_time': array([ 0.01256444]), 'split1_test_score': array([ 0.56431535]), 'split3_test_score': array([ 0.55879292]), 'mean_train_score': array([ 0.55065509]), 'mean_test_score': array([ 0.54135495]), 'rank_test_score': array([1], dtype=int32), 'mean_fit_time': array([ 0.48707695])}
GREP_ME***Best params of [AdaBoostClassifier] estimator run are
{}
GREP_ME***Best score of [AdaBoostClassifier] estimator

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    1.0s finished


GREP_ME***Results of [BaggingClassifier] estimator run are
{'std_score_time': array([ 0.00094395]), 'params': ({},), 'split0_test_score': array([ 0.61968912]), 'split4_train_score': array([ 0.98312565]), 'std_train_score': array([ 0.00224002]), 'split4_test_score': array([ 0.58958333]), 'mean_score_time': array([ 0.00858321]), 'split3_train_score': array([ 0.98493898]), 'split0_train_score': array([ 0.98102417]), 'split2_test_score': array([ 0.6018711]), 'split1_train_score': array([ 0.98492723]), 'split2_train_score': array([ 0.98779221]), 'std_test_score': array([ 0.01045096]), 'std_fit_time': array([ 0.00977235]), 'split1_test_score': array([ 0.59647303]), 'split3_test_score': array([ 0.59417274]), 'mean_train_score': array([ 0.98436165]), 'mean_test_score': array([ 0.60037406]), 'rank_test_score': array([1], dtype=int32), 'mean_fit_time': array([ 0.44499011])}
GREP_ME***Best params of [BaggingClassifier] estimator run are
{}
GREP_ME***Best score of [BaggingClassifier] estimator run

...

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   17.0s finished


GREP_ME***Results of [NuSVC] estimator run are
{'std_score_time': array([ 0.04655399]), 'params': ({},), 'split0_test_score': array([ 0.29740933]), 'split4_train_score': array([ 1.]), 'std_train_score': array([ 0.]), 'split4_test_score': array([ 0.303125]), 'mean_score_time': array([ 0.72340498]), 'split3_train_score': array([ 1.]), 'split0_train_score': array([ 1.]), 'split2_test_score': array([ 0.2952183]), 'split1_train_score': array([ 1.]), 'split2_train_score': array([ 1.]), 'std_test_score': array([ 0.00318327]), 'std_fit_time': array([ 0.10185411]), 'split1_test_score': array([ 0.30186722]), 'split3_test_score': array([ 0.30280957]), 'mean_train_score': array([ 1.]), 'mean_test_score': array([ 0.30008313]), 'rank_test_score': array([1], dtype=int32), 'mean_fit_time': array([ 5.11246448])}
GREP_ME***Best params of [NuSVC] estimator run are
{}
GREP_ME***Best score of [NuSVC] estimator run are
0.30008312552

***Starting [DecisionTreeClassifier] estimator run 
Fitting 5 folds for ea

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.2s finished


GREP_ME***Results of [DecisionTreeClassifier] estimator run are
{'std_score_time': array([  9.41225097e-05]), 'params': ({},), 'split0_test_score': array([ 0.51917098]), 'split4_train_score': array([ 1.]), 'std_train_score': array([ 0.]), 'split4_test_score': array([ 0.55]), 'mean_score_time': array([ 0.00168943]), 'split3_train_score': array([ 1.]), 'split0_train_score': array([ 1.]), 'split2_test_score': array([ 0.51039501]), 'split1_train_score': array([ 1.]), 'split2_train_score': array([ 1.]), 'std_test_score': array([ 0.0136111]), 'std_fit_time': array([ 0.00420853]), 'split1_test_score': array([ 0.52074689]), 'split3_test_score': array([ 0.5182102]), 'mean_train_score': array([ 1.]), 'mean_test_score': array([ 0.52369077]), 'rank_test_score': array([1], dtype=int32), 'mean_fit_time': array([ 0.08464131])}
GREP_ME***Best params of [DecisionTreeClassifier] estimator run are
{}
GREP_ME***Best score of [DecisionTreeClassifier] estimator run are
0.523690773067

***Starting [ExtraTree

[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.1s finished


In [14]:
sorted(results.items(), key=lambda x:x[1])

[]

In [69]:
print (errors)

{'RadiusNeighborsClassifier': JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.4/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.4/dist-packages/ipykernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.4/runpy.py in _run